In [1]:
import openai
import os
import json
import re
import pandas as pd
from datetime import datetime
from openai import OpenAI

In [2]:
from util_functions import get_completion_from_messages, extract_json_from_response 
from util_functions import retrieve_df_list_from_csv, parse_raw_csv, assign_category_and_subcategory

/Users/jiamingpei/Desktop/Projects/gptFinance/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from dataframe_configs import log_filename, log_schema_types, log_schema_cols, log_date_cols

In [4]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [5]:
file_list = retrieve_df_list_from_csv()

File 1: Date range.CSV loaded with Status         2
Date           2
Description    2
Debit          1
Credit         1
dtype: int64 records
File 2: DFS-Search-20240923.csv loaded with Trans. Date    4
Post Date      4
Description    4
Amount         4
Category       4
dtype: int64 records
File 4: Chase4739_Activity20240901_20240915_20240924.CSV loaded with Transaction Date    18
Post Date           18
Description         18
Category            17
Type                18
Amount              18
Memo                 0
dtype: int64 records
File 5: Credit Card - 0283_09-01-2024_09-15-2024.csv loaded with Date           6
Transaction    6
Name           6
Memo           6
Amount         6
dtype: int64 records
File 6: CreditCard1.csv loaded with 09/13/2024          3
98.55               3
*                   3
Unnamed: 3          0
Bill Pay Payment    3
dtype: int64 records


In [6]:
aggregate_book = parse_raw_csv(client, file_list)

In [7]:
category_map = assign_category_and_subcategory(client, aggregate_book)

In [9]:
cate_book = pd.DataFrame.from_records(category_map)
pre_audit_df = cate_book.sort_values("Date").reset_index(drop=True)
pre_audit_df.insert(0, "ID", pre_audit_df.index)
pre_audit_df['Date'] = pd.to_datetime(pre_audit_df['Date'])
pre_audit_df.insert(2, "Year", pre_audit_df['Date'].dt.year)
pre_audit_df.insert(3, "Month", pre_audit_df['Date'].dt.month)


pre_audit_output = pre_audit_df.copy()
pre_audit_output['Note'] = ""

if 'audit.csv' not in os.listdir(os.path.join(os.getcwd(), "output")):
    pre_audit_output.to_csv("output/audit.csv", index=False)
    print('Audit data saved.')
else:
    print('audit.csv already exist. Not saving a new copy.')

pre_audit_df = pre_audit_df.astype(log_schema_types)
pre_audit_df

Audit data saved.


,ID,Date,Year,Month,Merchant,Amount,Category,Subcategory
0,0,2024-08-30,2024,8,SHELL OIL 10008333006,66.68,Travel,Auto
1,1,2024-09-01,2024,9,SQ *THE KEBAB SHOP SANTA CLARA CA,22.97,Consumables,Dining
2,2,2024-09-02,2024,9,TAIWAN PORRIDGE MILPITAS CA,56.56,Consumables,Dining
3,3,2024-09-02,2024,9,BILTPROTECT RENT ACH CREDIT,3172.52,Home,Rent
4,4,2024-09-03,2024,9,99 RANCH #1766 MILPITAS CA,25.59,Consumables,Grocery
5,5,2024-09-03,2024,9,YANG GUO FU MALATANG,38.37,Consumables,Dining
6,6,2024-09-04,2024,9,ATT*BILL PAYMENT 800-288-2020 TX,65.35,Home,Utilities
7,7,2024-09-04,2024,9,NETFLIX.COM NETFLIX.COM CA,22.99,Subscription,Subscription
8,8,2024-09-05,2024,9,PANERA BREAD #204481 O,6.76,Consumables,Dining
9,9,2024-09-06,2024,9,TRADER JOE S #229 MILPITAS CA,24.13,Consumables,Grocery


In [10]:
"""
After manual audit
"""

audit_df = pd.read_csv(os.path.join(os.getcwd(), 'output/audit.csv'))
audit_df["ID"] = audit_df.index
audit_df['Date'] = pd.to_datetime(audit_df['Date'])
audit_df['Year'] = audit_df['Date'].dt.year
audit_df['Month'] = audit_df['Date'].dt.month
audit_df

,ID,Date,Year,Month,Merchant,Amount,Category,Subcategory,Note
0,0,2024-08-30,2024,8,SHELL OIL 10008333006,66.68,Travel,Auto,NaN
1,1,2024-09-01,2024,9,SQ *THE KEBAB SHOP SANTA CLARA CA,22.97,Consumables,Dining,NaN
2,2,2024-09-02,2024,9,TAIWAN PORRIDGE MILPITAS CA,28.28,Consumables,Dining,NaN
3,3,2024-09-02,2024,9,TAIWAN PORRIDGE MILPITAS CA,28.28,Consumables,Dining,Dating
4,4,2024-09-03,2024,9,99 RANCH #1766 MILPITAS CA,25.59,Consumables,Grocery,NaN
5,5,2024-09-03,2024,9,YANG GUO FU MALATANG,19.19,Consumables,Dining,NaN
6,6,2024-09-03,2024,9,YANG GUO FU MALATANG,19.19,Consumables,Dining,Dating
7,7,2024-09-04,2024,9,ATT*BILL PAYMENT 800-288-2020 TX,65.35,Home,Utilities,NaN
8,8,2024-09-04,2024,9,NETFLIX.COM NETFLIX.COM CA,22.99,Subscription,Subscription,NaN
9,9,2024-09-05,2024,9,PANERA BREAD #204481 O,6.76,Consumables,Dining,NaN


In [11]:
audit_df.to_csv("output/audit.csv", index=False)

In [13]:
# if log_filename not in os.listdir(os.path.join(os.getcwd(), 'output')):
#     empty_df = pd.DataFrame({col: pd.Series(dtype=dtype) for col, dtype in log_schema_types.items()})
#     for c in log_schema_cols:
#         if c not in empty_df.columns:
#             empty_df[c] = ""
#     empty_df = empty_df[log_schema_cols]
#     empty_df.to_csv(f"output/{log_filename}", index=False)

# old_logs = pd.read_csv(f"output/{log_filename}", dtype=log_schema_types, parse_dates=list(log_date_cols.keys()))

# audit_df['LogDate'] = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

# for k, v in log_date_cols.items():
#     audit_df[k] = pd.to_datetime(audit_df[k], format=v)

# new_logs = pd.concat([old_logs, audit_df])
# new_logs = new_logs.sort_values(['LogDate', 'Date']).reset_index(drop=True)
# new_logs["ID"] = new_logs.index

# threshold = 5
# if len(old_logs) > 0:
#     day_diff = (audit_df['LogDate'].max() - old_logs['LogDate'].max()).days
# else:
#     day_diff = threshold
# if day_diff < threshold:
#     user_res = input(f"The most recent log update is less than {day_diff} days. Do you want to update logs anyways?(y/n)")
#     if user_res.lower() == 'y':
#         new_logs.to_csv(f"output/{log_filename}", index=False)
#         print(f"Update successful. {len(audit_df)} new records inserted to the log.")
#     else:
#         print("Update aborted. No new records inserted to the log.")
# else:
#     new_logs.to_csv(f"output/{log_filename}", index=False)
#     print(f"Update successful. {len(audit_df)} new records inserted to the log.")

The most recent log update is less than 0 days. Do you want to update logs anyways?(y/n) y


Update successful. 90 new records inserted to the log.
